<a href="https://colab.research.google.com/github/architb1703/Toxic_Span/blob/archit/Span_Avg_F1_Metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==2.6.0
!pip install seqeval
!pip install urllib3 --upgrade
!pip install pytorch-crf

     |████████████████████████████████| 542kB 12.1MB/s 
     |████████████████████████████████| 133kB 38.7MB/s 
     |████████████████████████████████| 1.1MB 56.7MB/s 
     |████████████████████████████████| 3.7MB 18.7MB/s 
     |████████████████████████████████| 890kB 52.3MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 7.1MB 50.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=35645ecf25bfa6d3db9b13577d808bee419c32251995622bc0508e5226e21ba6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.34 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=084746556281de1b6cc2ea85c39

In [ ]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm, trange

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torchtext import data
import transformers
from transformers import BertTokenizer, BertConfig, BertForTokenClassification, AdamW, BertModel

from torchcrf import CRF
from transformers import get_linear_schedule_with_warmup

from keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix

torch.manual_seed(12)
np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Load data
train_path = '/content/drive/MyDrive/ToxicSpan_CS669V/BERT_Preprocess/train.pkl'
val_path = '/content/drive/MyDrive/ToxicSpan_CS669V/BERT_Preprocess/val.pkl'

# train_path = '/content/drive/MyDrive/ToxicSpan_CS669V/processed/finaltrain.pkl'
# val_path = '/content/drive/MyDrive/ToxicSpan_CS669V/processed/finaldev.pkl'

with open(train_path, 'rb') as f:
  train_data = pickle.load(f)
  f.close()

with open(val_path, 'rb') as f:
  val_data = pickle.load(f)
  f.close()

In [ ]:
#Class to take in tokenizer and model along with input data and calculate the span average f1 score
class SpanAvgF1():
  def __init__(self, model, tokenizer, X, y, spans, target_spans, maxlen=500):
    self.model = model
    self.model.cuda()
    self.tokenizer = tokenizer
    self.maxlen = maxlen
    self.X = X
    self.y = y
    self.spans = spans
    self.target_spans = target_spans
    self.prepare_data()

    self.X = torch.tensor(self.X)
    self.y = torch.tensor(self.y)
    self.attention_mask = torch.tensor(self.attention_mask)

    data = TensorDataset(self.X, self.attention_mask, self.y)
    self.dataloader = DataLoader(data, batch_size=16, shuffle=False)
  
  def tokenize_data(self, x, y, s):
    sentence = []
    labels = [0]
    spans = []
    for i in range(len(x)):
      word = x[i]
      label = y[i]
      tokenized_word = self.tokenizer.tokenize(word)
      sentence.extend(tokenized_word)
      labels.extend([label for k in range(len(tokenized_word))])
      curr = s[i][0]
      spans.append([curr, curr+len(tokenized_word[0])])
      curr += len(tokenized_word[0])
      for j in range(len(tokenized_word)-1):
        spans.append([curr, curr+len(tokenized_word[j+1])-2])
        curr += len(tokenized_word[j+1])-2
      spans[-1][-1] = s[i][1]
    labels.append(0)
    return(sentence, labels, spans)

  def get_attention_mask(self, x):
    return([[(i!=0) for i in text] for text in x])

  def prepare_data(self):
    for i in range(len(self.X)):
      self.X[i], self.y[i], self.spans[i] = self.tokenize_data(self.X[i], self.y[i], self.spans[i])
    self.X = pad_sequences([tokenizer.encode(text) for text in self.X], maxlen = self.maxlen, dtype='long', value=0.0, truncating='post', padding = 'post')
    self.y = pad_sequences(self.y, maxlen=self.maxlen, value=2, dtype='long', truncating='post', padding='post')
    self.attention_mask = self.get_attention_mask(self.X)
  
  def get_text_lengths(self, masks):
    lengths = []
    for mask in masks:
      lengths.append(torch.sum(mask).item())
    return(lengths)

  def evaluate(self, flag):
    predictions , true_labels = [], []
    for batch in self.dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_id, b_input_mask, b_labels = batch
      self.model.eval()
      
      if(flag==1):
        with torch.no_grad():
          outputs = self.model(b_input_id, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs[1].detach().cpu().numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      
      elif(flag==2):
        with torch.no_grad():
          out = model(b_input_id, b_input_mask)
          logits = torch.argmax(F.softmax(out, dim=2), dim=2).to('cpu').numpy()
          predictions.extend(logits)
      
      else:
        preds = model(b_input_id, b_input_mask, None, True)
        predictions.extend([p for p in preds])

      true_labels.extend(b_labels)
    if(flag!=3):
      self.pred_tags = [[p_i for p_i, l_i in zip(p, l) if l_i != 2][1:-1] for p, l in zip(predictions, true_labels)]
    else:
      self.pred_tags = predictions

  def f1score(self):
    self.f1list = []
    self.predicted_spans = []
    f1 = 0
    for i in range(len(self.spans)):
      s = [0 for k in range(len(val_data['text'][i]))]
      prev = 0
      for j in range(len(self.spans[i])):
        for k in range(self.spans[i][j][0], self.spans[i][j][1]):
          s[k] = self.pred_tags[i][j]
        if(prev==1 and self.pred_tags[i][j]==1):
          for l in range(self.spans[i][j-1][1], self.spans[i][j][0]):
            s[l] = 1
        
        prev = self.pred_tags[i][j]
      self.f1list.append(f1_score(self.target_spans[i], s))
      f1 += f1_score(self.target_spans[i], s, zero_division=1)
      self.predicted_spans.append(s)
    return(f1/len(self.X))

In [ ]:
X = val_data['token_final'].values
Y = val_data['target_final'].values
spans = val_data['span_final'].values
target_spans = []
for i in range(len(X)):
  s = [0 for j in range(len(val_data['text'][i]))]
  for k in val_data['spans'][i]:
    s[k] = 1
  target_spans.append(s)

In [ ]:
class BiLSTM(nn.Module):
  def __init__(self, bert, input_dim, num_labels, hidden_dim, lstm_layers, dropout):
    super().__init__()
    self.input_dim = input_dim
    self.num_labels = num_labels
    self.hidden_dim = hidden_dim
    self.bert = bert
    self.bilstm = nn.LSTM(input_dim, hidden_dim, lstm_layers, bidirectional=True, dropout=dropout, batch_first=True)
    self.fc = nn.Linear(hidden_dim*2, num_labels)

  def forward(self, inputs, masks, labels=None, bilstm=False):
    outputs = self.bert(inputs, masks)
    seq_out = outputs[0]
    
    seq_out = nn.utils.rnn.pack_padded_sequence(seq_out, torch.tensor([torch.sum(a) for a in masks]), batch_first=True, enforce_sorted=False)
    x, (h_n, c_n) = self.bilstm(seq_out)
    x, _ = nn.utils.rnn.pad_packed_sequence(x, total_length=300, batch_first=True)
    x = self.fc(x)
    return(x)

In [ ]:
class BertCRFModel(nn.Module):
  def __init__(self, bert_model, num_labels, bilstm):
    super(BertCRFModel, self).__init__()
    self.bert_model = bert_model
    self.num_labels = num_labels
    self.bert = BertModel.from_pretrained(self.bert_model, output_attentions=False, output_hidden_states=False)
    self.crf = CRF(self.num_labels, batch_first=True)
    self.dropout = nn.Dropout(0.7)
    if(bilstm):
      self.bilstm = nn.LSTM(self.bert.config.hidden_size, self.num_labels, bidirectional=True, num_layers=1, batch_first=True)
      self.fc = nn.Linear(self.num_labels*2, self.num_labels)
    else:
      self.fc = nn.Linear(self.bert.config.hidden_size, self.num_labels)
    
  def forward(self, inputs, masks, labels=None, bilstm=False):
    outputs = self.bert(inputs, masks)
    seq_out = outputs[0]
    
    if(not bilstm):
      x = self.fc(seq_out)
      seq_out = self.dropout(seq_out)
    else:
      seq_out = nn.utils.rnn.pack_padded_sequence(seq_out, torch.tensor([torch.sum(a) for a in masks]), batch_first=True, enforce_sorted=False)
      x, (h_n, c_n) = self.bilstm(seq_out)
      x, _ = nn.utils.rnn.pad_packed_sequence(x, total_length=500, batch_first=True)
      x = self.fc(x)

    masks = masks.type(torch.uint8)
    if(labels is not None):
      loss = -self.crf(F.log_softmax(x, dim=2), labels, mask=masks, reduction='mean')
      preds = self.crf.decode(x, mask=masks)
      return loss, preds
    else:
      preds = self.crf.decode(x, mask=masks)
      return preds

In [ ]:
model = torch.load('/content/drive/MyDrive/semi_sup-5.pt', map_location=torch.device('cpu'))
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case = False)

In [ ]:
metric = SpanAvgF1(model, tokenizer, X, Y, spans, target_spans, 500)
metric.evaluate(1)
metric.f1score()

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.6882755826103494

In [ ]:
metric.pred_tags[4], val_data['target_final'][4], val_data['token_final'][4]

([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 ['Good',
  'points',
  'A',
  'dumb',
  'crude',
  'guy',
  'in',
  'a',
  'dumb',
  'crude',
  'sport',
  'That',
  "'",
  's',
  'what',
  'Trump',
  'has',
  'done',
  'to',
  'politics',
  'I'])

In [ ]:
df = pd.DataFrame(list(zip(val_data['token_final'], metric.pred_tags)), 
               columns =['token_final', 'target_final'])
df.to_csv('/content/drive/My Drive/ToxicSpan_CS669V/processed/val_semi_new.csv')

In [ ]:
#Generating csv file to compare ground truth and predicted spans
text = []
truth_text = []
pred_text = []

for i in range(len(val_data['text'])):
  text.append(val_data['text'][i])
  arr = []
  for j,k in enumerate(val_data['target_final'][i][1:-1]):
    if(k == 1):
      arr.append(val_data['token_final'][i][j])
  truth_text.append(arr)
  arr = []
  # x = tokenizer.convert_ids_to_tokens(metric.X[i])
  for j,k in enumerate(metric.pred_tags[i]):
    if(k == 1):
      arr.append(val_data['token_final'][i][j])
  pred_text.append(arr)

In [ ]:
df = pd.DataFrame(list(zip(text, truth_text, pred_text)), 
               columns =['Text', 'Ground Truth', 'Predicted'])
df.to_csv('/content/drive/My Drive/ToxicSpan_CS669V/processed/semi_sup_it3.csv')